In [1]:
from licheats import Client
client = Client()

In [2]:
player = client.get_player('Fieber69')
player

In [3]:
games = client.get_games('Fieber69')
games

In [4]:
client.save_player(player)
client.save_games(games)

In [5]:
len(games)

79

In [6]:
client.resume_stats(player, games)

DetachedInstanceError: Instance <Player at 0x7fc961549290> is not bound to a Session; attribute refresh operation cannot proceed (Background on this error at: https://sqlalche.me/e/20/bhk3)